Weather data comes from swish analytics https://swishanalytics.com/mlb/weather?date=2022-11-05

Helpful website for understanding tables: https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

Read in game dates from GameLogs.csv

In [27]:
columns = ['Date','VisitingTeam','HomeTeam']
dates = pd.read_csv('source_files/gameLogsFixed.csv', skipinitialspace=True, usecols=columns)
dates['Date'] = pd.to_datetime(dates['Date'])
dates = dates[dates['Date'] > '2012-04-04'] # I went to the website and this is the first day with the weather data
dates['Date'] = dates['Date'].astype(str)
gameDays =  pd.Series(dates['Date']).drop_duplicates().tolist() # drop duplicates

#### Create List of URLs ####
swishAnalytics = 'https://swishanalytics.com/mlb/weather?date='
urlList = []
for games in gameDays:
    urlList.append(swishAnalytics + games)

In [24]:
index1 = 0
index2 = 1000
urlList2 = urlList[index1:index2]

# set up dataframe
website = []
date = []
startTime = [] # time game started in military time (ET)
visiting = []
home = []
avgTemp = [] # avg feels like
totalPrecip = [] # total % precipitation during the game
avgWindSpeed = []

headers = {"Accept-Language": "en-US, en;q=0.5"}

for url in urlList2:
        results = requests.get(url, headers=headers)
        soup = BeautifulSoup(results.text,"html.parser")
        weather_div = soup.find_all('div', class_ = 'weather-card')

        ### Loop through each game played that day ###
        for container in weather_div:
                #### Website Link ####
                website.append(url)

                #### Date ####
                date.append(url[-10:])

                #### Team Names ####
                lineup = container.h4.text
                # Visiting #
                team1 = lineup[:lineup.index('@')]
                visiting.append(team1)
                # Home #
                team2 = lineup[lineup.index('@')+2:]
                home.append(team2)

                #### Time ####
                time_div = soup.find('div', class_ = 'col-md-3 pad-5')
                time = container.small.text.strip(' ET')
                meridian = time[-2:] # "AM" or "PM"
                time_without_meridian = time[:-2]
                hour = int(time[:(time.index(':'))])
                if meridian == "PM" and hour != 12:
                        hour += 12
                        time = str(hour) + time_without_meridian[time.index(':'):]
                elif meridian == "AM" and hour ==12:
                        time = '0' + time_without_meridian[time.index(':'):]
                else: # for AM and noon
                        time = time_without_meridian
                startTime.append(time)

        #### Weather ####
        # Loop through all weather cards #
        temp_table = soup.find_all('table', class_ = 'table table-bordered')
        for table in temp_table:
                # This itterates through the table by row #  
                for row in table.tbody.find_all('tr'):
                        columns = row.find_all('td')
                        if(columns != []):
                                rowName = columns[0].text
                                ## Feels Like ##
                                if rowName == 'Feels Like':
                                        feelsLikeList = []
                                        for hour in columns:
                                                feelsLikeList.append(hour.text.strip('.°'))
                                        avgFeelsLike = np.mean([int(i) for i in feelsLikeList[1:]])
                                        avgTemp.append(avgFeelsLike)

                                ## Precipitation ##
                                if rowName == 'Precip %':
                                        precipList = []
                                        for hour in columns:
                                                precipList.append(hour.text.strip('%'))
                                        sumPrecip = np.sum([int(i) for i in precipList[1:]]) / 100
                                        totalPrecip.append(sumPrecip)

                                ## Wind ##
                                if rowName == 'Wind Speed':
                                        windList = []
                                        for hour in columns:
                                                windList.append(hour.text.strip(' mph'))
                                        avgWind = np.mean([int(i) for i in precipList[1:]])
                                        avgWindSpeed.append(avgWind)

weather_df = pd.DataFrame({
'url': website,
'date': date,
'homeTeam': home,
'visitingTeam':visiting,
'gameTime':startTime,
'avgTemp':avgTemp,
'percipitation':totalPrecip,
'avgWindSpeed':avgWindSpeed
})

print(weather_df)


# saving the dataframe
csv_name = 'weather' + str(index1) + '_' + str(index2) + '.csv'
weather_df.to_csv((csv_name), header=True, index=False, mode='w+')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [25]:
weather_df = pd.DataFrame({
'url': website,
'date': date,
'homeTeam': home,
'visitingTeam':visiting,
'gameTime':startTime,
'avgTemp':avgTemp,
'percipitation':totalPrecip,
'avgWindSpeed':avgWindSpeed
})
print(weather_df)

csv_name = 'weather' + str(index1) + '_' + str(index2) + '.csv'
weather_df.to_csv((csv_name), header=True, index=False, mode='w+')

                                                     url        date  \
0      https://swishanalytics.com/mlb/weather?date=20...  2012-04-05   
1      https://swishanalytics.com/mlb/weather?date=20...  2012-04-05   
2      https://swishanalytics.com/mlb/weather?date=20...  2012-04-05   
3      https://swishanalytics.com/mlb/weather?date=20...  2012-04-05   
4      https://swishanalytics.com/mlb/weather?date=20...  2012-04-05   
...                                                  ...         ...   
11249  https://swishanalytics.com/mlb/weather?date=20...  2012-06-04   
11250  https://swishanalytics.com/mlb/weather?date=20...  2012-06-04   
11251  https://swishanalytics.com/mlb/weather?date=20...  2012-06-04   
11252  https://swishanalytics.com/mlb/weather?date=20...  2012-06-04   
11253  https://swishanalytics.com/mlb/weather?date=20...  2012-06-04   

              homeTeam   visitingTeam gameTime  avgTemp  percipitation  \
0             Tigers        Red Sox      13:05   36.750      

In [26]:
csv_name = 'weather' + str(index1) + '_' + str(index2) + '.csv'
weather_df.to_csv((csv_name), header=True, index=False, mode='w+')